## SageMaker training with FSx

This sample shows how to:

- Setup FSx
- Associate data in S3 with FSx
- Run a SageMaker Training job using data from FSx mount
- Save artifacts into FSx which are automatically pushed to S3
- Tear down the infrastructure

Checkout this [blog](https://aws.amazon.com/blogs/machine-learning/choose-the-best-data-source-for-your-amazon-sagemaker-training-job/) to verify if FSx is needed for your use-case to save operational costs. 

**Please make sure the CIDR block in setup/cfn-nlp.yaml does not conflict with your existing VPC. You can also change FSx storage (currently set at 1.2 TB) depending on your data sets**

In [60]:
# Imports
import os
import time
import boto3
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
from sagemaker.inputs import FileSystemInput

# Clients
cfn_client = boto3.client("cloudformation", region_name=region)
fsx_client = boto3.client("fsx", region_name=region)

# Inputs
region = "us-east-1"  # update this if your region is different
region_az = "us-east-1c"  # customize this as needed. Your FSx will be set up in a subnet in this AZ
cfn_stack_name = 'fsx-training'  # cloudformation stack name

s3_data_bucket = 's3://sagemaker-us-east-1-988346548731'  # s3 bucket for training artifacts and datasets
s3_data_train_prefix = 'train'  # s3 training data set
s3_data_model_prefix = 'model_dir' # s3 path to save model
s3_data_checkpoint_prefix = 'checkpoint_dir'  # s3 path to save model checkpoints
fsx_file_system_path = 'fsx-train'  # this is file system path on FSx for the data, can be anything

In [28]:
# Setup infrastructure using CloudFormation
with open("setup/cfn-nlp.yaml", "r") as f:
    template_body = f.read()
    
create_stack_response = cfn_client.create_stack(
    StackName=cfn_stack_name,
    TemplateBody=template_body,
    Parameters=[
        {
            'ParameterKey': 'AZ',
            'ParameterValue': region_az
        }
    ]
)

create_stack_response

In [37]:
# Wait for stack to be created, it takes ~10 minutes to complete.
stack_id = create_stack_response['StackId']

while True:
    response = cfn_client.describe_stacks(
        StackName=stack_id
    )
    status = response['Stacks'][0]['StackStatus']
    if status== "CREATE_IN_PROGRESS":
        print("Create in progress. Waiting..")
        time.sleep(30)
    elif status=="CREATE_COMPLETE":
        print("Stack created!")
        break
    else:
        print("Error creating stack - check the CFN console")
        break

Create in progress. Waiting..
Create in progress. Waiting..
Create in progress. Waiting..
Create in progress. Waiting..
Create in progress. Waiting..
Create in progress. Waiting..
Stack created!


In [57]:
# Get stack outputs
describe_response = cfn_client.describe_stacks(
    StackName=stack_id
)

outputs = describe_response['Stacks'][0]['Outputs']

for output in outputs:
    if output['OutputKey'] == 'sg':
        sec_group = output['OutputValue']
    elif output['OutputKey'] == 'outputfsx':
        fsx_id = output['OutputValue']
    elif output['OutputKey'] == 'privatesubnet':
        private_subnet_id = output['OutputValue']
        
fsx_response = fsx_client.describe_file_systems(
    FileSystemIds=[fsx_id]
)

fsx_mount = fsx_response['FileSystems'][0]['LustreConfiguration']['MountName']

print("FSx ID:", fsx_id)
print("Security Group ID:", sec_group)
print("Private Subnet ID:", private_subnet_id)
print("FSx Mount path:", fsx_mount)

FSx ID: fs-049c156b4adff4308
Security Group ID: sg-01e94965efb7c618d
Private Subnet ID: subnet-0b14dd61ba3bba5f5
FSx Mount path: tb6ttbev


In [77]:
# Create a data repository association with S3 to load data
# and persist changes back to S3 to save training artifacts

fsx_s3_response = fsx_client.create_data_repository_association(
    FileSystemId=fsx_id,
    FileSystemPath=f"/{fsx_file_system_path}",
    DataRepositoryPath=s3_data_bucket,
    BatchImportMetaDataOnCreate=True,
    S3={
        "AutoImportPolicy": {
            "Events": ['NEW', 'CHANGED', 'DELETED']
        }
    }
)

fsx_s3_response

{'Association': {'AssociationId': 'dra-01a6a11ae830552b4',
  'ResourceARN': 'arn:aws:fsx:us-east-1:988346548731:association/fs-049c156b4adff4308/dra-01a6a11ae830552b4',
  'FileSystemId': 'fs-049c156b4adff4308',
  'Lifecycle': 'CREATING',
  'FileSystemPath': '/fsx-train',
  'DataRepositoryPath': 's3://sagemaker-us-east-1-988346548731',
  'BatchImportMetaDataOnCreate': True,
  'ImportedFileChunkSize': 1024,
  'S3': {'AutoImportPolicy': {'Events': ['NEW', 'CHANGED', 'DELETED']}},
  'Tags': [],
  'CreationTime': datetime.datetime(2022, 5, 4, 22, 24, 25, 513000, tzinfo=tzlocal())},
 'ResponseMetadata': {'RequestId': '9a331141-1aa8-4d46-a5e9-1b86edc2babc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 04 May 2022 22:24:25 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '486',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9a331141-1aa8-4d46-a5e9-1b86edc2babc'},
  'RetryAttempts': 0}}

In [78]:
# Wait for association to be complete
while True:
    fsx_s3_assoc = fsx_client.describe_data_repository_associations(
        AssociationIds=[fsx_s3_response['Association']['AssociationId']]
    )
    fsx_status = fsx_s3_assoc['Associations'][0]['Lifecycle']

    if fsx_status== "CREATING":
        print("Create in progress. Waiting..")
        time.sleep(30)
    elif fsx_status=="AVAILABLE":
        print("FSx - S3 association complete!")
        break
    else:
        print("Error creating the association, with status", fx_status)
        breaksec_group

Create in progress. Waiting..
Create in progress. Waiting..
Create in progress. Waiting..
Create in progress. Waiting..
Create in progress. Waiting..
Create in progress. Waiting..
Create in progress. Waiting..
Create in progress. Waiting..
Create in progress. Waiting..
Create in progress. Waiting..
Create in progress. Waiting..
Create in progress. Waiting..
Create in progress. Waiting..
Create in progress. Waiting..
FSx - S3 association complete!


Now, we create a SageMaker training job that uses FSx as the training input. For detailed parameters for training job, see the [CreateTrainingJob API](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateTrainingJob.html).

In [79]:
# Setup fsx config for data channels
fsx_directory_path = f"/{fsx_mount}/{fsx_file_system_path}"

fsx_input = FileSystemInput(
    file_system_id=fsx_id,
    file_system_type='FSxLustre',
    directory_path=fsx_directory_path,
    file_system_access_mode="rw", # write needed for saving model artifacts to fsx
)
data_channels = {"train": fsx_input}

# for ease, so that you can use fsx for data and training artifacts
SM_TRAIN_DIR = "/opt/ml/input/data/train"  # path where fsx is mounted in the training container
hyperparameters = {}
hyperparameters["checkpoint-dir"] = f"{SM_TRAIN_DIR}/{s3_data_checkpoint_prefix}"
hyperparameters["model-dir"] = f"{SM_TRAIN_DIR}/{s3_data_model_prefix}"
hyperparameters["training-dir"] = f"{SM_TRAIN_DIR}/{s3_data_train_prefix}"

In [80]:
# setup estimator and invoke
instance_type = "ml.m5.xlarge"
instance_count = 1
base_job_name = f'sagemaker-fsx-mount-sample'

estimator = PyTorch(
    entry_point="train.py",
    source_dir=os.getcwd(),
    instance_type=instance_type,
    role=get_execution_role(),
    instance_count=instance_count,
    framework_version="1.8.1",
    py_version="py36",
    checkpoint_s3_uri=None,  # as it is FSx
    checkpoint_local_path=hyperparameters["checkpoint-dir"],  # FSx
    hyperparameters=hyperparameters,
    base_job_name=base_job_name,
    subnets = [private_subnet_id], # Give SageMaker Training Jobs access to FSx resources in your Amazon VPC
    security_group_ids=[sec_group],
    max_retry_attempts=30)

estimator.fit(inputs=data_channels)

2022-05-04 22:32:28 Starting - Starting the training job...
2022-05-04 22:32:52 Starting - Launching requested ML instancesProfilerReport-1651703548: InProgress
.........
2022-05-04 22:34:12 Starting - Preparing the instances for training......
2022-05-04 22:35:13 Downloading - Downloading input data
2022-05-04 22:35:13 Training - Downloading the training image...
2022-05-04 22:35:58 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-05-04 22:35:59,154 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-05-04 22:35:59,157 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-05-04 22:35:59,168 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-05-04 22:35:59,175 sagemaker_pytorch_container.training INFO     Invoking user training scrip

### Clean up resources

You can tear down the CloudFormation stack to delete the VPC and associated resources, and the FSx file system to avoid incurring costs.

In [81]:
# Delete the stack

delete_response = cfn_client.delete_stack(
    StackName=stack_id
)

delete_response

{'ResponseMetadata': {'RequestId': '514e896f-cb7a-4b65-9a92-d726d986b5ed',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '514e896f-cb7a-4b65-9a92-d726d986b5ed',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Wed, 04 May 2022 22:39:07 GMT'},
  'RetryAttempts': 0}}